In [40]:
import numpy as np
import matplotlib.pyplot as plt
import math
import control as co
from control.matlab import *
from scipy import signal
import numpy as np
from numpy.linalg import eig
from IPython.display import Image
from IPython.core.display import HTML 
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


##### ![logo](img/upp2.png)
# Universidad Politécnica de Puebla 
# JJVS 2025
# Control moderno

## Representación en variables de estado 

Los sistemas dinámicos lineales, se pueden representar con variables de estado:

\begin{equation}
  \dot x=A \cdot x+ B \cdot u 
\end{equation} 

\begin{equation}
  y=C \cdot x
\end{equation} 


El punto sobre el vector $x$ representa la derivada de las variables de estado. 

## Modelado matemático de un motor de CD
Un motor de CD se modela considerando las partes eléctrica y mecánica. Como se sabe un motor esta compuesto de un embobinado que presenta una resistencia (R) e inductancia (L). Por otra parte se genera un voltaje contraelectromotriz $e(t)$ que está en función de la velocidad del motor $\omega$ y el flujo electromagnético $\phi$. Para modelar la parte mecánica se considera el coeficiente de inercia del eje del motor *J* y el par de carga $T_L$.

<center><img src="img/motorCDv20.png" width="500px"> <center>

## Representación en variables de estado del motor de CD

La representación del motor de CD, con $T_L=0$ es:

\begin{equation}
	 \begin{bmatrix}
	  di(t)/{dt} \\
	  d\omega(t)/{dt}
	 \end{bmatrix}
	 =
	 \begin{bmatrix}
	 -R/L & -K/L\\
	 K/J &-b/J	 
	 \end{bmatrix}
	 \begin{bmatrix}
	  i(t) \\
	 \omega(t)
	 \end{bmatrix}
	 +
	 \begin{bmatrix}
	 1/L\\
	 0 
	 \end{bmatrix}
	 v(t)
\end{equation}

Haciendo un cambio de variable, $x_1=i(t)$ y $x_2=\omega(t)$, se obtiene:

\begin{equation}
	 \begin{bmatrix}
	  \dot{x}_1 \\
	  \dot{x}_2 
	 \end{bmatrix}
	 =
	 \begin{bmatrix}
	 -R/L & -K/L\\
	 K/J &-b/J	 
	 \end{bmatrix}
	 \begin{bmatrix}
	  x_1(t) \\
	 x_2(t)
	 \end{bmatrix}
	 +
	 \begin{bmatrix}
	 1/L\\
	 0 
	 \end{bmatrix}
	 v(t)
\end{equation}

$$\dot x=A \cdot x+ B \cdot u $$


#### Considere los siguientes valores del motor:

(**J**)     moment of inertia of the rotor     **3.2284E-6 kg.m^2 <p>**
(**b**)     motor viscous friction constant    **3.5077E-6 N.m.s  <p>**
(**Kb**)    electromotive force constant       **0.0274 V/rad/sec <p>**
(**Kt**)    motor torque constant              **0.0274 N.m/Amp <p>**
(**R**)     electric resistance                **4 Ohm <p>**
(**L**)     electric inductance                **2.75E-6H <p>**

La representación en python de las matrices, con Kt=Kb=K, es la siguiente:

In [41]:
J = 3.2284E-6; b = 3.5077E-6; K = 0.0274; R = 4;
L = 2.75E-6;

A=np.array([[-R/L, -K/L],[K/J, -b/J]])
B=np.array([[1/L],[0]])
C=[0,1]
D=0

In [42]:
print("A=",A)

A= [[-1.45454545e+06 -9.96363636e+03]
 [ 8.48717631e+03 -1.08651344e+00]]


In [43]:
display(B)

array([[363636.36363636],
       [     0.        ]])

In [44]:
display(C)

[0, 1]

In [45]:
w,v=eig(A)
print('E-value:', w)

E-value: [-1.45448732e+06 -5.92260385e+01]


## Convertidor CD-CD (buck)

Las ecuaciones de estado del convertidor CD-CD, reductor o buck.


<center><img src="img/buck.png" width="400px"> <center>


<center><img src="img/buck_circ.png" width="400px"> <center>

Son las siguientes:

\begin{equation}
	 \begin{bmatrix}
	  \dot{x}_1 \\
	  \dot{x}_2 
	 \end{bmatrix}
	 =
	 \begin{bmatrix}
	 0 & -1/L\\
	 1/C &-1/(R*C)	 
	 \end{bmatrix}
	 \begin{bmatrix}
	  x_1(t) \\
	 x_2(t)
	 \end{bmatrix}
	 +
	 \begin{bmatrix}
	 E/L\\
	 0 
	 \end{bmatrix}
	 u(t)
\end{equation}

Y la matriz $C$ será:
\begin{equation}
C=
 \begin{bmatrix}
  0 ~ 1
 \end{bmatrix}
\end{equation} 


## Linealización

Algunos sistemas son no lineales, para esos casos se emplean las siguientes matrices para la linealización alrededor de un punto de equilibrio:

\begin{equation}
A^*=
\begin{bmatrix}
\partial{f_1}/\partial{x_1} &\partial{f_1}/{\partial{x_2}} &\partial{f_1}/{x_3}\\
\partial{f_2}/\partial{x_1} &\partial{f_2}/{\partial{x_2}} &\partial{f_2}/{x_3}\\
\partial{f_3}/\partial{x_1} &\partial{f_3}/{\partial{x_2}} &\partial{f_3}/{x_3}\\
\end{bmatrix}
\end{equation}

\begin{equation}
B^*=
\begin{bmatrix}
\partial{f_1}/\partial{u}\\
\partial{f_2}/\partial{u}\\
\partial{f_3}/\partial{u}
\end{bmatrix}
\end{equation}

Donde $f_1$ es la primera ecuación obtenida del modelado del sistema, $f_2$ la segunda ecuación, $f_3$ la tercera ecuación y $u$ la entrada.


**Ejemplo 1**

a) Obtener las ecuaciones que describan el comportamiento dinámico del sistema.

b) Linealizar el modelo.

<center><img src="img/levmagv2.png" width="400px"><center>

Las ecuaciones del sistema son las siguientes:

\begin{equation}
\dot x_1=x_2\\
\dot x_2=g-x_3^2/x_1\\
\dot x_3=-\frac{R}{L}x_3+e(t)
\end{equation}

Linealizando el sistema, se obtiene:

\begin{equation}
\begin{bmatrix}
\dot x_1\\
\dot x_2\\
\dot x_3
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 & 0\\
\frac{x_{30}^2}{m x_{10}^2} & 0 &-\frac{2x_{30}}{m x_{10}}  \\
0 & 0 & -\frac{R}{L}
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3
\end{bmatrix}
+
\begin{bmatrix}
0\\
0\\
-\frac{1}{L}
\end{bmatrix}
e(t)
\end{equation}

En el punto de equilibrio, es decir cuando el sistema ha alcanzado la estabilidad se obtiene de la segunda ecuación de (9) 
\begin{equation}
x_{30}=\sqrt {g x_{10}}
\end{equation}

Sustituyendo en (10) se tiene:

\begin{equation}
\begin{bmatrix}
\dot x_1\\
\dot x_2\\
\dot x_3
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 & 0\\
\frac{g}{x_{10}} & 0 &-2\sqrt\frac{g}{mx_{10}}  \\
0 & 0 & -\frac{R}{L}
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3
\end{bmatrix}
+
\begin{bmatrix}
0\\
0\\
-\frac{1}{L}
\end{bmatrix}
e(t)
\end{equation}



## Control mediante retroalimentación de estados
A los sistemas representados por variables de estados, se les puede aplicar un controlador por retroalimentacion de estados, sin y con un integrador. Veamos el primer caso, sin integrador. Considere el sistema definido por las ecuaciones de estado

\begin{equation}
  \dot x=A \cdot x+ B \cdot u 
\end{equation} 

\begin{equation}
  y=C \cdot x
\end{equation} 

	
Si se aplica la transformada de Laplace a la ecuación anterior se tiene:
\begin{equation}
sX(s)=AX(s)+BU(s)
\end{equation}
	
Despejando $X(s)$, se tiene:
\begin{equation}
sX(s)-AX(s)=BU(s)
\end{equation}

Y factorizando:
\begin{equation}
[sI-A]X(s)=BU(s)
\end{equation}
	
y despejando $X(s)$ se tiene:
\begin{equation}
	X(s)=[sI-A]^{-1}BU(s)
\end{equation}

De la ecuación anterior se tiene que la ecuación característica, el denominador se iguala a cero:

\begin{equation}
    |sI-A|=0
\end{equation}

donde los eigenvalores del determinante, son los polos del sistema.

Si las variables de estado $x$ se retroalimentan, es decir se multiplican por una constante cada una de ellas y se conectan a la entrada, se obtiene el siguiente diagrama a bloques. Al hacer la retroalimentación de las variables de estados, ahora se tiene que $u=r(t)-Kx(t)$. 

<center><img src="img/retross2.png" width="600px"> <center>

Por lo que ahora la nueva ecuación de estado es:
\begin{equation}
  \dot x=A \cdot x + B \cdot (r(t)-K \cdot x(t))
\end{equation} 

\begin{equation}
  y=C \cdot x
\end{equation} 



	
Si se aplica la transformada de Laplace a la ecuación, anterior se tiene:
\begin{equation}
sX(s)=AX(s)+BR(s)-BKX(s)
\end{equation}
	
Despejando $X(s)$, se tiene:
\begin{equation}
sX(s)-AX(s)+BKX(s)=BR(s)
\end{equation}
 
despejando $X(s)$
\begin{equation}
X(s)=\frac{BR(s)}{sI-A+BK}
\end{equation}

 y la salida $y$, será

 \begin{equation}
 Y(S)=CX(s)=\frac{CBR(s)}{sI-A+BK}
 \end{equation}


Y la ecuación característica del sistema retroalimentado será:
\begin{equation}
|sI-A+BK|=0
\end{equation}

Para este caso los eigenvalores (polos) del sistema dependen de los valores de $K$. Es decir, la selección de estas constantes $K$, permitirán reubicar los polos del sistema. Es importante recordar que el sistema es estable si los polos se situan a la izquierda del plano $S$.

### **Controlabilidad**
Para aplicar el control por retroalimentación de estados el sistema debe ser controlable, esto se cumple sí y solo sí, la matriz 
\begin{equation}
{S}=[B\ \; AB \; A^2B \; \ldots \; A^{n-1}B]
\end{equation}
es de rango n. 
	
### **Definición: Rango de una matriz**
Es el orden del mayor determinante distinto de cero que se puede encontrar en el interior de una matriz determinada.
	
 
#### **Teoremas de retroalimentación de estados**
* Una condición necesaria y suficiente para la colocación arbitraria de los polos es que el sistema sea de estado completamente controlable.
* Si el par [**A,B**] es completamente controlable, existe una matriz ${K}$ que puede dar un conjunto arbitrario de valores característicos de $({A}-{B}{K})$; las $n$ raíces de la ecuación característica se obtienen de los eigenvalores de:

\begin{equation}
|s{I}-{A}+{B}{K}|
\end{equation}



**Ejercicio 1**
Determine si los siguientes sistemas son controlables.

\begin{eqnarray}
\begin{bmatrix}
\dot{x1} \\ \dot{x2}
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 \\ 0 & -1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
+
\begin{bmatrix}
1  \\ 0
\end{bmatrix}
u
\end{eqnarray}
	
\begin{eqnarray}
\begin{bmatrix}
\dot{x1} \\ \dot{x2}
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 \\ 2 & -1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
+
\begin{bmatrix}
0  \\ 1
\end{bmatrix}
u
\end{eqnarray}


**Ejercicio 2**

Diseñe un control por retroalimentación de estados para el siguiente sistema. 

<center><img src="img/buck.png" width="400px"> <center>

Se desea que los polos se ubiquen en s=-3000 $\pm$ j3000. Con E=12V, L=1mH, C=100$\mu$F, R=12.5$\Omega$. Emplee el comando:

**K=ppol(A,B,polos)**

para obtener la matriz de ganancias K en función de la ubicación de los polos en Scilab.

In [46]:
L=1e-3; C=100e-6; R=8.2;   E=12; 

A=[[0, -1/L],[1/C, -1/(R*C)]]
B=[[E/L],[0]]
C=[0,1]
D=0

p = [-3000-3000j, -3000+3000j]
K=co.place(A, B, p)
K

matrix([[0.39837398, 0.01808447]])

**Ejercicio 3**

Realizar un control por retroalimentacion de estados para el sistema de suspensión magnética, seleccione los polos en $s_1=-100$, $s_2=-100-100j$ y $s_3=-100+100j$. Obtenga sus conclusiones y muestre la simulación.
\begin{eqnarray}
A&=&
\begin{bmatrix}
0 & 1 & 0\\
x_{03}^2/mx_{01}^2 & 0 & -2x_{03}/mx_{01}\\
0 & 0 & -R/L
\end{bmatrix}\\
B&=&\begin{bmatrix}
0\\
0\\
1/L\\
\end{bmatrix}\\
C&=& \begin{bmatrix}
1~ & ~0 & 0\\
\end{bmatrix}
\end{eqnarray}
Con $x_{03}=\sqrt{mgx_{01}}$, $R = 0.5\Omega$, $L = 10mH$; $m = 20mg$; $g = 9.8m/s^2$; $x_{01} = 0.03 m$. 



In [47]:
R=0.5; L = 10e-3; m = 20e-3; g = 9.8; x01 = 0.03;x03 = math.sqrt(m*g*x01);
A=[[0,1,0],[x03**2/(m*x01**2),0,-2*x03/(m*x01)],[0,0,-R/L]]
B=[[0],[0],[1/L]]
C=[1,0,0]
D=0

## Retroalimentación de estados con control integral
Para eliminar el error en estado estable se agrega un integrador y por otra parte se agrega la señal de referencia ($r(t) \neq 0$). De tal manera que aumentan en uno las variables de estado.

<center><img src="img/retroint.png" width="500px"> <center>
    
\begin{eqnarray}
\begin{bmatrix}
\dot{ {x}(t)}\\
\dot{x_{n+1}}\\
\end{bmatrix}
=\begin{bmatrix}
 {A} & 0\\
- {C} & 0\\
\end{bmatrix}
\begin{bmatrix}
 {x}(t)\\
x_{n+1}\\
\end{bmatrix}
+
\begin{bmatrix}
 {B}\\
 {-D}\\
\end{bmatrix}
u(t)
+
\begin{bmatrix}
 {0}\\
1\\
\end{bmatrix}
r(t)
\end{eqnarray}



ya que $u=-Kx-K_{n+1}x_{n+1}$

\begin{equation}
	\begin{bmatrix}
		\dot {x(t)}\\
		\dot{x_{n+1}}\\
	\end{bmatrix}
	=
	\begin{bmatrix}
		 A & 0\\
		- C & 0\\
	\end{bmatrix}
	\begin{bmatrix}
		 x(t)\\
		x_{n+1}\\
	\end{bmatrix}
	-
	\begin{bmatrix}
		 B\\
		 -D\\
	\end{bmatrix}
	\begin{bmatrix}
		 K & k_{n+1}\\
	\end{bmatrix}
	\begin{bmatrix}
		 x(t)\\
		x_{n+1}\\
	\end{bmatrix}
	+
	\begin{bmatrix}
		 0\\
		1\\
	\end{bmatrix}
	r(t)
\end{equation}
	


La ecuación de estado puede expresarse como:


\begin{equation}
\dot{\hat{ {x}}}=\hat{ {A}}\hat{ {x}}-\hat{ {B}} \hat{ {K}} \hat{ {x}}+
\begin{bmatrix}
 {0}\\
1\\
\end{bmatrix}
r(t)
\end{equation}
 


Y la ecuación característica para el sistema es:

\begin{equation}
|s{\hat I}-\hat{A}+\hat{B}\hat{K}|=0
\end{equation}

donde:


\begin{eqnarray}
\hat{ {A}}=
\begin{bmatrix}
 {A} & 0\\
- {C} & 0\\
\end{bmatrix}, 
\hat{ {B}}=\begin{bmatrix}
 {B}\\
 {-D}\\
\end{bmatrix},
\hat{ {K}}=
\begin{matrix}
[k_1 & k_2 & k_3 \:  ... \: & k_{n+1}]
\end{matrix}
\end{eqnarray}
 


**Ejercicio 4**				

Realice el controlador para el reductor (buck) para que los polos se ubiquen en -3000, -3000+j3000 y -3000-j3000. En Scilab representar el circuito y el controlador.

In [48]:
R=8.2; C=100e-6; L=1e-3; E=12; 

Aa=[[0, -1/L,0],[1/C, -1/(R*C),0],[0,-1,0]]
Ba=[[E/L],[0],[0]]
C=[0,1]
D=0

pa = [-3000-3000j, -3000+3000j, -3000]
K=co.place(Aa, Ba, pa)
K

matrix([[ 6.48373984e-01,  1.37596669e-01, -4.50000000e+02]])

**Ejercicio 5**

Diseñe un control por retroalimentación de estados con integrador para que el sistema de levitación magnética para que los polos se ubique en:  -100, -10, -150+j150, -150-j150. Emplee Scilab.

In [49]:
R=0.5; L = 10e-3; m = 20e-3; g = 9.8; x01 = 0.03;x03 = math.sqrt(m*g*x01);
Ag=[[0,1,0,0],[x03**2/(m*x01**2),0,-2*x03/(m*x01),0],[0,0,-R/L,0],[-1,0,0,0]]
Bg=[[0],[0],[1/L],[0]]
Cg=[1,0,0,0]
Da=0

In [50]:
p = [-100-100j, -100+100j,-100,-50]
K=co.place(Ag, Bg, p)
K

matrix([[-1.60965227e+02, -2.16454738e+00,  3.00000000e+00,
          3.91230398e+03]])

## **Observabilidad**
Un sistema es completamente observable si cada variable de estado del sistema afecta alguna de las salidas.
	
#### Teorema
Para que el sistema descrito por las ecuaciones: 

\begin{eqnarray}
\dot x = A \cdot  x + B \cdot u \\
y={C} \cdot x + D \cdot u
\end{eqnarray}

sea completamente observable es necesario y suficiente que la matriz:
\begin{eqnarray}
V=
\begin{bmatrix}
C & CA & CA^2 & \: ... \:CA^{n-1}
\end{bmatrix}
^T
\end{eqnarray}

sea de rango **n**.

**Ejercicio 6**

Determine si los siguientes sistemas son observables.
\begin{eqnarray}
A=
\begin{bmatrix}
-2 & 0 \\
0 & -1 \\ 
\end{bmatrix} 
B=
\begin{bmatrix}
3  \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
1 & 0 
\end{bmatrix} 
\end{eqnarray}
	
\begin{eqnarray}
A=
	\begin{bmatrix}
	0 & 1 \\
	-2 & -3 \\ 
	\end{bmatrix} 
	B=
	\begin{bmatrix}
	1  \\
	1  \\
	\end{bmatrix} \:
	C=
	\begin{bmatrix}
	1 & 2 
	\end{bmatrix} 
	\end{eqnarray}
	

 

In [51]:
A = [[-2, 0], [0, -1]]
C = [1, 0]
O1=co.obsv(A,C)

In [52]:
O1

matrix([[ 1.,  0.],
        [-2.,  0.]])

In [53]:
A = [[0, 1], [-2, -3]]
C = [[1, 2]]
O2=co.obsv(A,C)

In [54]:
O2

matrix([[ 1.,  2.],
        [-4., -5.]])

Para el sistema de levitación magnética calcule la observabilidad, si la salida es la posición, la velocidad o la corriente.

## **Observador de orden completo**
Para un sistema representado con variables de estado, se puede obtener un observador de orden completo de la siguiente forma:

\begin{equation}
  \dot {\tilde x} = A \tilde x + B u + k_e ( y - C \tilde x)
\end{equation}

<center><img src="img/obscom.png" width="400px"> <center>

El problema de diseñar un observador de orden completo está en determinar la matriz de ganancias del observador ${K}_e$ de forma que la dinámica del error sea asintóticamente estable con una velocidad de respuesta suficiente. \\
	
Restando el sistema original del observador se tiene:
\begin{equation}
\dot x - \dot {\tilde x} = {A}({x} - {\tilde {x}}) -{K}_e  C (x - {\tilde x})
\end{equation}

La cual puede reescribirse en términos del error como:
\begin{equation}
\dot{e}=(A-K_eC)e
\end{equation}

Los valores propios deseados de la ecuación característica se deben escoger de forma que el observador de estados responda de 2 a 5 veces más rápido que el sistema en lazo cerrado. La ecuación característica del observador es:
\begin{equation}
|s{I}-{A}+{K}_e {C}|=0
\end{equation}


**Ejercicio 7**

Considere el sistema representado mediante variables de estado, con:
\begin{eqnarray}
A=
\begin{bmatrix}
0 & 20.6 \\
1 & 0 \\ 
\end{bmatrix} 
\:
B=
\begin{bmatrix}
0 \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
0 & 1 
\end{bmatrix} 
\end{eqnarray}
	
Determine la matriz de ganancias ${K}_e$ tal que los valores propios deseados de la matriz del observador sean $\gamma _1 =-5$ y $\gamma _2 =-5$. Considere que $u=-K \tilde x$.


**Ejercicio 8**

Considere el diseño de un regulador para el sistema representado mediante variables de estado, con:
\begin{equation}
A=
\begin{bmatrix}
0 & 1 \\
20.6 & 0 \\ 
\end{bmatrix} 
\:
B=
\begin{bmatrix}
0 \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
1 & 0 
\end{bmatrix} 
\end{equation}

Los polos en lazo cerrado del sistema se desean en $s_{12}=-1.8 \pm j1.8$. Suponga un control por realimentación del estado observado.\\

\begin{equation}
    u=-K\tilde{x}
\end{equation}

con polos del observador en $s_1=-8$ y $s_2=-8$. Encuentre $K$ y $K_e$.


In [55]:
A = [[0, 20.6], [1, 0]]
B = [[0], [1]]
C = [0, 1]
p = [-5, -4.99]
co.place(A, B, p)

matrix([[2.21116505, 9.99      ]])

## **Observador de orden mínimo**
Algunas de las variables se pueden estimar a partir de una señal que puede ser medida; en otras palabras a partir de la señal medida se observan las otras señales. Considerando el caso en el que solo una variable de estado es medible, para el sistema:

\begin{equation}
    \dot{x} = A \cdot  x + B \cdot u \\
    y= C \cdot x + D \cdot u
\end{equation} 
 
La ecuación de estado puede reescribirse como:

\begin{equation}
    \dot{x}=
    \begin{bmatrix}
    \dot{x_a} \\
    \dot{{x}_b}
    \end{bmatrix}
    \begin{bmatrix}
    A_{aa} & {A}_{ab} \\
    {A}_{ba} & {A}_{bb}
    \end{bmatrix}
    \begin{bmatrix}
    x_a \\
    {x}_b
    \end{bmatrix}
    +
    \begin{bmatrix}
    B_a\\
    {B}_b
    \end{bmatrix}u
\end{equation}
	
\begin{eqnarray}
    y=
    \begin{bmatrix}
    1&0
    \end{bmatrix}
    \begin{bmatrix}
    x_a \\
    {x}_b
    \end{bmatrix}
\end{eqnarray}

de donde:
    
\begin{eqnarray}
\dot{x_a}=A_{aa}x_a+{A}_{ab}{x}_b+B_au\\
\dot{{x}_b}={A}_{ba}x_a+{A}_{bb}{x}_b+{B}_bu
\end{eqnarray}
 

 
\begin{eqnarray}
\dot{x_a}=A_{aa}x_a+{A}_{ab}{x}_b+B_au\\
\dot{{x}_b}={A}_{ba}x_a+{A}_{bb}{x}_b+{B}_bu
\end{eqnarray}

Con
\begin{equation}
y^*=\dot{x_a}-A_{aa}x_a-B_au\\ 
{C}^*={A}_{ab}\\ 
{B}^*u^*= {A}_{ba}x_a+{B}_bu
\end{equation}
Se tiene
\begin{eqnarray}
\dot{{x}_b}&=&{A}_{bb}{x}_b+{B}^*u^*\\
y^*&=&{C}^*{x}_b 
\end{eqnarray}
 

Sustituyendo con los cambios de variables realizados, se tiene: 
\begin{eqnarray}
\dot{{x}_b}&=&{A}_{bb}{x}_b+{A}_{ba}x_a+{B}_bu\\
\dot{{x}_b}&=&{A}_{bb}{x}_b+{B}^*u^*\\
\dot{x_a}-A_{aa}x_a-B_au&=&{A}_{ab}{x}_b\\% y^*&=&{C}^*{x}_b \\
\end{eqnarray}
	
Ahora considere la ecuación del observador de orden completo
\begin{equation}
\dot {\tilde x} = {A} \tilde x + {B}u+{K}_e(y- {C} \tilde x) = ({A}-{K}_e C )\tilde x + {B}u+ {K}_e y
\end{equation}

Haciendo cambios de variables 
$\tilde x =\tilde{{x}_b}$, ${A}={A}_{bb}$, ${C}={A}_{ab}$, ${B}{u}= {A}_{ba}x_a+{B}_bu$ y $y=\dot{x_a}-A_{aa}x_a-B_au$.
 

Después de sustituir se obtiene la ecuación:

\begin{equation}
\tilde { \dot x_b }=( A_{bb}-{K}_e A_{ab})\tilde {{x}_b}+ {A}_{ba}x_a+{B}_bu+{K}_e(\dot{x_a}-A_{aa}x_a-B_au)
\end{equation}
	
Al estimar $\tilde{ x_b}$ se requiere la derivada de $x_a$, pero si ésta señal es ruidosa, la derivada amplificaría el ruido. Para evitar la derivada se hace el cambio de variable $\tilde{\dot x_b}=\dot Z +{K}_e \dot{x_a}$ y se obtiene:
	
\begin{equation}
\dot{Z}=(A_{bb}-{K}_e A_{ab})\tilde {x_b}+ A_{ba} x_a + B_bu- K_e(A_{aa}x_a+B_au)
\end{equation}
	
Agrupando términos se tiene:

\begin{equation}
\dot{Z}=({A}_{bb}-{K}_e{A}_{ab}){Z}+ [({A}_{bb}- {K}_e{A}_{ab}){K}_e +{A}_{ba} -{K}_e Aaa]y  + ({B}_b-{K}_e{B}_a)u 
\end{equation}

Para obtener las variables observadas, es decir $\mathbf{\tilde{x}}_b$, se integra $\mathbf{Z}$ y se emplea la ecuación 

\begin{equation}
\tilde{x_b}=Z+K_e{x_a}
\end{equation}



**Ejercicio 9**

Para un motor de CD, con las siguientes parámetros :
$K_m=0.06$, $J_m=80.45e-6$, $L=0.003$, $R=3.13$, $B_m=20e-6$.

*  Obtenga su representación en variables de estado.
*  Diseñe un observador de orden mínimo para observar el par aplicado.
*  Seleccione los valores de la ubicación de los polos del observador y del motor (los valores empleados deben ser únicos, es decir no pueden repetir entre equipos).


## **Motor síncrono de imanes permanentes**

Este motor es empleado en vehículos híbridos y eléctricos, las ecuaciones que representan la dinámica del sistema son:

\begin{equation}
L_{d}\frac{di_{d}}{dt} = \omega L_{q}i_{q}P - Ri_{d} + v_{d}\\ 
%\label{mod_elec_d}
%L_{q}\frac{di_{q}}{dt} =   -\omega(L_{d}i_{d}P + K_m) - Ri_{q} + v_{q}
\end{equation}

\begin{equation}
L_{q}\frac{di_{q}}{dt} =   -\omega(L_{d}i_{d}P + K_m) - Ri_{q} + v_{q}
\end{equation}


\begin{equation}
\frac{2}{3}J\frac{d\omega}{dt} =K_mi_{q} - \frac{2}{3}(T_{L} + B\omega)
\end{equation} 

**Ejercicio 10**

Este sistema presenta un polo en el semiplano derecho y dos en el semiplano izquierdo, por lo tanto es inestable. Calcular las ganancias de la matriz $K$ para que los polos sean: (s+ 500),(s+ 500-j500),(s+ 500 +j500). Considere los siguientes valores:
$P=2$, $K_m = 0.17445$, $J=31\times10^{-6}~kgm^2$, $R=2.35 \Omega$, $L = 6.65 mH$; $B = 77.5\times 10^{-6}~ Nm/rad$. 



Considere la matriz $K$ de la siguiente manera:
\begin{equation}
K=
\begin{bmatrix}
k_1 & 0 & 0 \\
0&k_2&k_3
\end{bmatrix}
\end{equation}